## POINTWISE MUTUAL INFORMATION - Trigrams

In [1]:
import spacy
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import random
import pickle
from unidecode import unidecode
import nltk
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import skipgrams
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords 
from spacy.lang.en import English

from tqdm import tqdm
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures,TrigramCollocationFinder, TrigramAssocMeasures

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## PMI - DRUG

In [3]:
#Reload save pkl file
with open('drug.pkl', 'rb') as f:
    drug = pickle.load(f)

In [4]:
def get_pmi_value(word1, word2, matrix):
    return matrix.loc[matrix.index == word1, [word2]]

def pmi(df, positive=True):
    cols = df.sum(axis=0)
    total = cols.sum()
    rows = df.sum(axis=1)
    expected = np.outer(rows, cols) / total
    df_pmi = df / expected
    with np.errstate(divide='ignore'):
        df_pmi = np.log(df_pmi)
    df_pmi[np.array(np.isinf(df_pmi))] = 0.0 
    if positive:
        df_pmi[df_pmi < 0] = 0.0
    return df_pmi

def max_cooccurrences(df_column):
    return df_column.idxmax() , df_column.max()

In [5]:
vectorizer = CountVectorizer(min_df=0.01,  ngram_range=(3,3))
X = vectorizer.fit_transform(drug.Text)

In [6]:
pmi_drug = pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [7]:
asint = pmi_drug.astype(int)
drug_pmi = asint.T.dot(asint)
drug_pmi.values[tuple([np.arange(drug_pmi.shape[0])]*2)] = 0

In [8]:
drug_occ = pmi(drug_pmi, positive=True)
drug_occ

,guilty reasonable doubt,evidence sufficient sustain,sufficient sustain conviction,evidence favorable prosecution,favorable prosecution rational,prosecution rational essential,guilt reasonable doubt,reasonable doubt guilt,evidence closely balanced,contrary manifest evidence,...,recover damages sustained,unreasonable improbable unsatisfactory,murder armed robbery,doubt guilt evidence,evidence tending prove,ascertain effect intent,evidence aggravation mitigation,shall shall shall,conclusion foregoing affirm,guilty armed robbery
guilty reasonable doubt,0.000000,0.875149,0.664160,0.000000,0.000000,0.000000,0.911269,0.565069,0.000000,0.549603,...,0.000000,1.401759,0.000000,0.000000,1.206267,0.295599,0.000000,0.000000,0.956685,0.000000
evidence sufficient sustain,0.875149,0.000000,0.000000,0.642616,1.551458,1.399430,0.416611,0.000000,0.000000,0.902243,...,0.000000,2.670690,3.940324,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sufficient sustain conviction,0.664160,0.000000,0.000000,0.654770,0.424178,0.000000,1.121912,1.859399,0.000000,0.000000,...,0.000000,0.850262,1.532110,0.000000,0.000000,0.000000,0.000000,0.000000,2.014626,2.632452
evidence favorable prosecution,0.000000,0.642616,0.654770,0.000000,0.609610,0.526575,1.857390,1.066505,0.000000,2.309500,...,0.264692,0.000000,0.000000,0.000000,0.000000,0.180849,0.000000,0.000000,1.928124,0.504249
favorable prosecution rational,0.000000,1.551458,0.424178,0.609610,0.000000,0.000000,0.000000,0.000000,0.000000,1.408233,...,0.796240,0.468630,0.000000,0.000000,0.678603,0.748765,0.000000,0.000000,0.177706,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ascertain effect intent,0.295599,0.000000,0.000000,0.180849,0.748765,1.138334,0.000000,0.000000,0.000000,0.035011,...,0.626990,0.481702,0.000000,0.000000,1.944438,0.000000,0.000000,0.000000,0.000000,0.000000
evidence aggravation mitigation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.142325,0.029646,0.000000,0.000000,1.056613,0.000000,0.000000
shall shall shall,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.715658,0.000000,...,0.000000,0.000000,0.000000,0.350934,0.000000,0.000000,1.056613,0.000000,0.000000,0.000000
conclusion foregoing affirm,0.956685,0.000000,2.014626,1.928124,0.177706,1.393954,1.403263,2.069686,0.000000,0.000000,...,1.064932,0.960465,0.032875,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.329468


In [9]:
max_cooccurrences(drug_occ)

(guilty reasonable doubt                      reversed absent discretion
 evidence sufficient sustain                        murder armed robbery
 sufficient sustain conviction                   tested positive cocaine
 evidence favorable prosecution          determine credibility testimony
 favorable prosecution rational              evidence sufficient sustain
                                                    ...                 
 ascertain effect intent            improbable unsatisfactory reasonable
 evidence aggravation mitigation         evidence insufficient establish
 shall shall shall                       evidence insufficient establish
 conclusion foregoing affirm                        guilty armed robbery
 guilty armed robbery                        reasonable degree certainty
 Length: 83, dtype: object,
 guilty reasonable doubt            1.875406
 evidence sufficient sustain        3.940324
 sufficient sustain conviction      4.592488
 evidence favorable prosecution   

## PMI - WEAPONS

In [10]:
#Reload save pkl file
with open('weapons.pkl', 'rb') as f:
    weapons = pickle.load(f)

In [11]:
vectorizer = CountVectorizer(min_df=0.01, ngram_range=(3,3))
X = vectorizer.fit_transform(weapons.Text)

In [12]:
pmi_weapons = pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [13]:
asint = pmi_weapons.astype(int)
weapons_pmi = asint.T.dot(asint)
weapons_pmi.values[tuple([np.arange(weapons_pmi.shape[0])]*2)] = 0

In [14]:
weapons_occ = pmi(weapons_pmi, positive=True)
weapons_occ

,guilty degree murder,proven guilty reasonable,guilty reasonable doubt,prove guilty reasonable,evidence favorable rational,rational essential reasonable,essential reasonable doubt,reasonable doubt evidence,unsatisfactory reasonable doubt,reasonable doubt guilt,...,testimony testimony evidence,bodily forcible felony,conclusion foregoing affirm,armed violence statute,evidence tending prove,verdict manifest evidence,violence armed violence,felony armed violence,robbery armed weapon,shall provided shall
guilty degree murder,0.000000,0.323992,0.753101,0.000000,0.299945,0.000000,0.000000,0.186317,0.147465,0.000000,...,0.000000,0.000000,0.697357,0.000000,0.428398,0.0,0.000000,0.000000,0.000000,1.193021
proven guilty reasonable,0.323992,0.000000,0.000000,0.000000,0.017314,0.083018,1.095503,0.000000,0.298470,0.000000,...,0.477352,0.623675,0.560680,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.816630
guilty reasonable doubt,0.753101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.574836,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.652571,0.191890,0.876344,0.000000
prove guilty reasonable,0.000000,0.000000,0.000000,0.000000,0.000000,0.079446,0.000000,0.002392,0.000000,0.000000,...,0.000000,0.907784,0.000000,0.198232,0.729981,0.0,0.000000,0.552439,0.000000,0.000000
evidence favorable rational,0.299945,0.017314,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.047840,0.481845,0.000000,0.273068,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
verdict manifest evidence,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.118439,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.726848,0.0,0.000000,0.000000,0.000000,0.000000
violence armed violence,0.000000,0.000000,0.652571,0.000000,0.000000,0.000000,0.000000,0.197636,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.870686,2.083020,0.000000
felony armed violence,0.000000,0.000000,0.191890,0.552439,0.000000,0.000000,0.000000,0.295735,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.870686,0.000000,2.147840,0.000000
robbery armed weapon,0.000000,0.000000,0.876344,0.000000,0.000000,0.000000,0.000000,0.464025,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.083020,2.147840,0.000000,0.000000


In [15]:
max_cooccurrences(weapons_occ)

(guilty degree murder                 evidence sufficient prove
 proven guilty reasonable           disturbed absent discretion
 guilty reasonable doubt         sufficient preclude imposition
 prove guilty reasonable               degree murder conviction
 evidence favorable rational     conviction evidence improbable
                                             ...               
 verdict manifest evidence             degree murder conviction
 violence armed violence            reasonable doubt conviction
 felony armed violence          evidence aggravation mitigation
 robbery armed weapon           evidence aggravation mitigation
 shall provided shall                        guilty plea guilty
 Length: 250, dtype: object,
 guilty degree murder           2.740966
 proven guilty reasonable       1.857319
 guilty reasonable doubt        3.055922
 prove guilty reasonable        3.055473
 evidence favorable rational    2.954884
                                  ...   
 verdict manifest evi

## PMI - FINANCE

In [16]:
#Reload save pkl file
with open('finance.pkl', 'rb') as f:
    finance = pickle.load(f)

In [17]:
vectorizer = CountVectorizer(min_df=0.01,  ngram_range=(3,3))
X = vectorizer.fit_transform(finance.Text)

In [18]:
pmi_finance= pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [19]:
asint = pmi_finance.astype(int)
finance_pmi = asint.T.dot(asint)
finance_pmi.values[tuple([np.arange(finance_pmi.shape[0])]*2)] = 0

In [20]:
finance_occ = pmi(finance_pmi, positive=True)
finance_occ

,contrary manifest evidence,guilty reasonable doubt,evidence evidence evidence,breach fiduciary duty,shall shall shall,shall provided shall,provided shall shall,accordance expressed reversed,brought recover damages,evidence tending prove,evidence reversed reversed
contrary manifest evidence,0.000000,0.000000,0.000000,0.010134,0.633419,0.000000,0.000000,0.000000,0.000000,0.581037,0.634684
guilty reasonable doubt,0.000000,0.000000,0.000000,1.484945,0.000000,0.000000,0.000000,1.418360,0.000000,0.000000,0.000000
evidence evidence evidence,0.000000,0.000000,0.000000,0.000000,0.000000,1.638025,2.116763,0.000000,0.000000,0.000000,0.000000
breach fiduciary duty,0.010134,1.484945,0.000000,0.000000,0.729483,0.604841,0.000000,0.000000,0.000000,0.000000,0.000000
shall shall shall,0.633419,0.000000,0.000000,0.729483,0.000000,0.822661,1.210426,1.405993,0.000000,0.000000,0.000000
shall provided shall,0.000000,0.000000,1.638025,0.604841,0.822661,0.000000,1.825451,0.316270,0.000000,0.000000,0.000000
provided shall shall,0.000000,0.000000,2.116763,0.000000,1.210426,1.825451,0.000000,0.079881,0.000000,0.000000,0.000000
accordance expressed reversed,0.000000,1.418360,0.000000,0.000000,1.405993,0.316270,0.079881,0.000000,0.000000,0.000000,0.000000
brought recover damages,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.856840,1.062817
evidence tending prove,0.581037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.856840,0.000000,0.885136


In [21]:
max_cooccurrences(finance_occ)

(contrary manifest evidence          evidence reversed reversed
 guilty reasonable doubt                  breach fiduciary duty
 evidence evidence evidence                provided shall shall
 breach fiduciary duty                  guilty reasonable doubt
 shall shall shall                accordance expressed reversed
 shall provided shall                      provided shall shall
 provided shall shall                evidence evidence evidence
 accordance expressed reversed          guilty reasonable doubt
 brought recover damages             evidence reversed reversed
 evidence tending prove              evidence reversed reversed
 evidence reversed reversed             brought recover damages
 dtype: object,
 contrary manifest evidence       0.634684
 guilty reasonable doubt          1.484945
 evidence evidence evidence       2.116763
 breach fiduciary duty            1.484945
 shall shall shall                1.405993
 shall provided shall             1.825451
 provided shall shall 

## PMI - SEXUAL

In [22]:
#Reload save pkl file
with open('sexual.pkl', 'rb') as f:
    sexual = pickle.load(f)

In [23]:
vectorizer = CountVectorizer(min_df=0.01 ,ngram_range=(3,3))
X = vectorizer.fit_transform(sexual.Text)

In [24]:
pmi= pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [26]:
asint = pmi.astype(int)
_pmi = asint.T.dot(asint)

In [30]:
_pmi = pmi(_pmi, positive=True)

,manifest evidence decision,contrary manifest evidence,evidence manifest evidence,manifest evidence opposite,evidence opposite conclusion,opposite conclusion clearly,brought recover damages,foregoing reversed reversed,duty exercise reasonable,duty breach duty,...,provided shall shall,recover damages sustained,shall shall shall,decision contrary manifest,conclusion clearly evident,accordance expressed reversed,verdict notwithstanding verdict,evidence tending prove,shall provided shall,evidence guilty negligence
manifest evidence decision,4.978094,0.390637,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.135077,0.000000,0.000000,1.408023,0.000000,0.000000,0.000000
contrary manifest evidence,0.390637,4.742105,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.498837,0.722539,0.000000,0.000000,0.000000
evidence manifest evidence,0.000000,0.000000,2.106022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
manifest evidence opposite,0.000000,0.000000,0.000000,2.777275,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.864396,0.000000,0.000000,0.000000,0.000000,0.000000
evidence opposite conclusion,0.000000,0.000000,0.000000,0.000000,1.843269,0.000000,0.001967,0.000000,0.184623,0.000000,...,1.535407,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.197703,0.000000
opposite conclusion clearly,0.000000,0.000000,0.000000,0.000000,0.000000,1.192847,0.000000,1.121629,0.000000,0.000000,...,0.000000,0.000000,0.048234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
brought recover damages,0.000000,0.000000,0.000000,0.000000,0.001967,0.000000,2.021869,0.000000,0.000000,1.089924,...,0.000000,1.844572,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.351111,0.802697
foregoing reversed reversed,0.000000,0.000000,0.000000,0.000000,0.000000,1.121629,0.000000,2.042290,0.362429,0.000000,...,0.000000,0.000000,0.068442,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
duty exercise reasonable,0.000000,0.000000,0.000000,0.000000,0.184623,0.000000,0.000000,0.362429,1.776635,0.000000,...,0.214289,0.000000,0.362814,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
duty breach duty,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.089924,0.000000,0.000000,2.448329,...,0.000000,1.118633,0.000000,0.000000,0.010702,0.000000,0.000000,0.000000,0.000000,0.362146


## PMI - DIVORCE

In [31]:
#Reload save pkl file
with open('divorce.pkl', 'rb') as f:
    divorce = pickle.load(f)

In [32]:
vectorizer = CountVectorizer(min_df=0.01, ngram_range=(2,2))
X = vectorizer.fit_transform(divorce.Text)

In [33]:
pmi= pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [34]:
asint = pmi.astype(int)
_pmi = asint.T.dot(asint)
_pmi.values[tuple([np.arange(_pmi.shape[0])]*2)] = 0

In [36]:
_pmi = pmi(_pmi, positive=True)

,according evidence,decision decision,decision evidence,evidence insufficient,evidence contrary,contrary manifest,manifest evidence,shall shall,ordinary meaning,affidavit affidavit,...,sustaining demurrer,clearly expressed,competent testimony,shall testator,entirely different,testator gave,hear determine,leaving leaving,making appropriation,appropriation provided
according evidence,0.000000,0.472590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.419634,...,0.000000,0.000000,0.000000,0.000000,0.975600,0.000000,0.558421,0.000000,1.036476,0.000000
decision decision,0.472590,0.000000,0.000000,0.843213,0.153241,0.000000,0.518954,0.637081,0.863228,0.000000,...,0.000000,0.117624,0.364999,0.000000,1.187643,0.316659,0.000000,0.000000,0.000000,0.000000
decision evidence,0.000000,0.000000,0.000000,0.048650,0.000000,0.000000,0.823003,1.634277,0.000000,0.000000,...,0.000000,2.031111,2.614959,3.731623,2.051308,0.000000,0.669048,1.194658,0.000000,0.000000
evidence insufficient,0.000000,0.843213,0.048650,0.000000,0.822646,0.759182,0.000000,0.795660,0.000000,1.831711,...,0.000000,0.276203,0.000000,0.000000,0.247610,0.000000,1.216726,0.000000,0.000000,0.000000
evidence contrary,0.000000,0.153241,0.000000,0.822646,0.000000,0.000000,0.680709,1.204301,1.024983,0.000000,...,0.000000,1.195670,0.000000,0.000000,0.656251,0.000000,0.000000,1.745511,0.023980,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
testator gave,0.000000,0.316659,0.000000,0.000000,0.000000,0.000000,0.000000,0.269106,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.039174,1.735959,0.000000,0.000000,0.000000,0.000000,0.000000
hear determine,0.558421,0.000000,0.669048,1.216726,0.000000,0.974116,0.892467,0.000000,1.103210,0.000000,...,1.018587,0.000000,0.000000,0.000000,0.868009,0.000000,0.000000,0.858657,0.000000,0.000000
leaving leaving,0.000000,0.000000,1.194658,0.000000,1.745511,0.000000,0.000000,0.437591,0.886882,0.837653,...,0.000000,0.611281,0.000000,0.900807,0.988154,0.000000,0.858657,0.000000,0.355883,0.009676
making appropriation,1.036476,0.000000,0.000000,0.000000,0.023980,0.471341,0.000000,0.000000,0.000000,0.000000,...,1.208960,0.000000,0.000000,0.000000,0.365235,0.000000,0.000000,0.355883,0.000000,0.000000


### PMI - BURGLARY

In [37]:
#Reload save pkl file
with open('burglary.pkl', 'rb') as f:
    burglary = pickle.load(f)

In [38]:
vectorizer = CountVectorizer(min_df=0.01,  ngram_range=(3,3))
X = vectorizer.fit_transform(burglary.Text)

In [39]:
pmi= pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [40]:
asint = pmi.astype(int)
_pmi = asint.T.dot(asint)
_pmi.values[tuple([np.arange(_pmi.shape[0])]*2)] = 0

In [42]:
_pmi = pmi(_pmi, positive=True)

,contrary manifest evidence,sexual assault sexual,assault sexual assault,accordingly reversed reversed,evidence evidence evidence,plea guilty guilty,factual basis plea,evidence aggravation mitigation,prove reasonable doubt,deviate sexual assault,...,murder armed robbery,armed robbery armed,ascertain effect intent,reversed consistent reversed,minimum maximum sentence,armed robbery conviction,shall guilty misdemeanor,prove preponderance evidence,evidence sufficient sustain,evidence establish guilt
contrary manifest evidence,0.000000,0.841175,0.000000,0.447457,0.633160,0.000000,0.000000,2.059261,0.000000,0.000000,...,0.000000,0.665381,0.000000,0.000000,2.039159,0.000000,0.000000,0.000000,0.143728,0.000000
sexual assault sexual,0.841175,0.000000,1.845024,1.583401,0.188654,0.000000,0.222696,0.000000,1.840050,0.000000,...,0.637231,3.489901,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
assault sexual assault,0.000000,1.845024,0.000000,1.488577,0.000000,0.000000,0.094811,0.331271,3.294285,0.000000,...,0.000000,1.799592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
accordingly reversed reversed,0.447457,1.583401,1.488577,0.000000,0.000000,0.000000,0.000000,0.000000,1.805277,0.000000,...,0.000000,1.718547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
evidence evidence evidence,0.633160,0.188654,0.000000,0.000000,0.000000,0.827576,0.302363,1.832408,0.000000,0.000000,...,1.527828,0.446497,0.000000,1.068967,2.485251,0.000000,0.019178,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
armed robbery conviction,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.311038,1.050248,...,0.000000,0.000000,0.000000,0.000000,1.028453,0.000000,0.449449,0.223545,0.000000,0.000000
shall guilty misdemeanor,0.000000,0.000000,0.000000,0.000000,0.019178,0.000000,0.420944,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.449449,0.000000,1.420436,0.000000,0.000000
prove preponderance evidence,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.374964,0.223545,1.420436,0.000000,0.000000,0.000000
evidence sufficient sustain,0.143728,0.000000,0.000000,0.000000,0.000000,0.000000,1.040536,0.000000,0.000000,0.000000,...,1.176266,0.000000,1.383044,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.762297
